In [11]:
# port: 8900

In [19]:
import flask
import json
import librosa
import scipy
import scipy.io.wavfile as sio
import tempfile

def pcen(S, sr=22050, hop_length=512, gain=0.98, bias=2, power=0.5,
         time_constant=0.400, eps=1e-6, b=None, max_size=1, ref=None,
         axis=-1, max_axis=None):
    """
    Adapted from librosa
    """
    if b is None:
        t_frames = time_constant * sr / float(hop_length)
        # By default, this solves the equation for b:
        #   b**2  + (1 - b) / t_frames  - 2 = 0
        # which approximates the full-width half-max of the
        # squared frequency response of the IIR low-pass filter
        b = (np.sqrt(1 + 4 * t_frames**2) - 1) / (2 * t_frames**2)

    if np.issubdtype(S.dtype, np.complexfloating):
        S = np.abs(S)

    if ref is None:
        if max_size == 1:
            ref = S
        elif S.ndim == 1:
            raise ParameterError('Max-filtering cannot be applied to 1-dimensional input')
        else:
            if max_axis is None:
                if S.ndim != 2:
                    raise ParameterError('Max-filtering a {:d}-dimensional spectrogram '
                                         'requires you to specify max_axis'.format(S.ndim))
                # if axis = 0, max_axis=1
                # if axis = +- 1, max_axis = 0
                max_axis = np.mod(1 - axis, 2)

            ref = scipy.ndimage.maximum_filter1d(S, max_size, axis=max_axis)

    S_smooth = scipy.signal.lfilter([b], [1, b - 1], ref, axis=axis)

    # Working in log-space gives us some stability, and a slight speedup
    smooth = np.exp(-gain * (np.log(eps) + np.log1p(S_smooth / eps)))
    return (S * smooth + bias)**power - bias**power

def callback(data):
    spec = librosa.stft(data, n_fft=512, hop_length=128)
    spec = pcen(spec, sr=16000, hop_length=128)
    return librosa.feature.mfcc(S=spec, sr=16000, hop_length=128, n_mfcc=24).T

import keras
import keras.backend as K
import numpy as np
import tensorflow as tf

class PcenFeature():
    # class stub
    def __call__(self, recording):
        return callback(recording)


class ASR:

    _MEAN, _STD = 0.15655953, 1.3286684
    _model_path = "models/pcen_model.h5"
    _transform = PcenFeature()

    def __init__(self):
        self.mdl = keras.models.load_model(self._model_path, {
            "MEAN": self._MEAN, "STD": self._STD, "tf": tf})
        self.transform = self._transform
        self.mdl.predict(np.zeros([1, 1, self.mdl.input_shape[2]]))
    
    def predict(self, X):
        feats = np.stack([self.transform(X)])
        phonemes = self.mdl.predict(feats)
        return phonemes
        # map phonemes to words
        # use lattices

    def predict_phones(self, X):
        feats = np.stack([self.transform(X)])
        phonemes = self.mdl.predict(feats)
        trans = K.ctc_decode(phonemes, [phonemes.shape[1]], greedy=False, beam_width=2500, top_paths=5)
        trans = trans[0][0].eval(session=K.get_session())
        return [lang[x] for x in trans[0]]
        # map phonemes to words
        # use lattices

        
    
    
    def predict_words(self, X):
        # stub
        phoneme_likelihoods = self.mdl.predict(feats)
        

asr = ASR()
phonemes = asr.predict(rec)



app = flask.Flask(__name__)

lang = ['tS', 'dz', 'sil', 'en', 'e', 'I', 'v', 'S', 'i', 'r', 'dzi', 'on', 'f', 'n', 'si', 'o', 't', 'k', 'l', 'ts', 's', 'x', 'z', 'ni', 'b', 'j', 'zi', 'g', 'p', 'dZ', 'Z', 'tsi', 'a', 'd', 'm', 'w', 'u']

@app.route('/')
def api_index():
    # do this automatically
    return json.dumps({"endpoints": [
        {"url": "/recognize", "description": "main endpoint for recognition of phonemes"},
        {"url": "/probabilities", "description": "main endpoint for recognition of phonemes"},
        {"url": "/recognize", "description": "main endpoint for recognition of phonemes"}
    ]})

@app.route('/recognize', methods=["POST"])
def recognize():
    if 'file' not in request.files:
        return ("No file in the request", 400)
    file = request.files['file']
    if file.filename == '':
        return ("No file in the request", 400)
    if file:
        filename = tempfile.mktemp()
        try:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            rec = sio.read(filename)[1].astype(np.float32) / 2**15
            return asr.predict_phones(rec)
        finally:
            os.remove(filename)

@app.route('/probabilities')
def calculate_probability_matrices():
    if 'file' not in request.files:
        return ("No file in the request", 400)
    file = request.files['file']
    if file.filename == '':
        return ("No file in the request", 400)
    if file:
        filename = tempfile.mktemp()
        try:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return asr.predict(filename)
        finally:
            os.remove(filename)

@app.route('/phoneme_map')
def phoneme_map():
    return json.dumps(lang)

if __name__ == "__main__":
    app.run("0.0.0.0", 8900)


/venv/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 * Running on http://0.0.0.0:8900/ (Press CTRL+C to quit)


/venv/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


0.9655172413793104